In [7]:
import yaml
import os

config_file = "destination-rules.yaml"
kubectl_apply_cmd = "kubectl apply -f "

In [44]:
# https://istio.io/latest/docs/reference/config/networking/destination-rule/#ConnectionPoolSettings
class IstioConfig:
    def __init__(self, file_name):
        self.file_name = file_name
        with open(file_name, "r") as stream:
            docs = yaml.load_all(stream, Loader=yaml.FullLoader)
            self.docs = list(docs)
    
    # save to yaml file
    def save(self):
        with open(self.file_name, "w") as stream:
            yaml.dump_all(self.docs, stream)
    
    # apply to Istio cluster
    def apply(self):
        self.save()
        os.system(kubectl_apply_cmd + self.file_name)
        
    # reset to default config (does not save)
    def reset(self):
        for i in range(self.size()):
            ic.docs[i]["spec"]["trafficPolicy"]["connectionPool"]["http"] = {}
            ic.docs[i]["spec"]["trafficPolicy"]["connectionPool"]["tcp"] = {}
    
    def size(self):
        return len(self.docs)
    
    def get_docs(self):
        return self.docs
    
    def _get_http(self, i):
        return ic.docs[i]["spec"]["trafficPolicy"]["connectionPool"]["http"]
    
    def _get_tcp(self, i):
        return ic.docs[i]["spec"]["trafficPolicy"]["connectionPool"]["tcp"]
    
    #### tcp ####
    # maxConnections: int32
    def set_tcp_max_connections(self, i, val):
        self._get_tcp(i)["maxConnections"] = val
    
    # connectTimeout: int (unit ms)
    def set_tcp_connect_timeout(self, i, val):
        self._get_tcp(i)["connectTimeout"] = str(val) + "ms"
        
    # TcpKeepalive time: int (unit ms)
    def set_tcp_keeplive_probes(self, i, val):
        val = str(val) + "ms"
        tcp = self._get_tcp(i)
        if "tcpKeepalive" in tcp:
            tcp["tcpKeepalive"]["time"] = val
        else:
            tcp["tcpKeepalive"] = {"time": val}
            
    # TcpKeepalive time: int (unit ms)
    def set_tcp_keeplive_time(self, i, val):
        val = str(val) + "ms"
        tcp = self._get_tcp(i)
        if "tcpKeepalive" in tcp:
            tcp["tcpKeepalive"]["time"] = val
        else:
            tcp["tcpKeepalive"] = {"time": val}
            
    # TcpKeepalive interval: int (unit ms)
    def set_tcp_keeplive_interval(self, i, val):
        val = str(val) + "ms"
        tcp = self._get_tcp(i)
        if "tcpKeepalive" in tcp:
            tcp["tcpKeepalive"]["interval"] = val
        else:
            tcp["tcpKeepalive"] = {"interval": val}
    
    #### http ####
    # http1MaxPendingRequests: int32
    def set_http_http1_max_pending_requests(self, i, val):
        self._get_http(i)["http1MaxPendingRequests"] = val
    
    # http2MaxRequests: int32
    def set_http_http2_max_requests(self, i, val):
        self._get_http(i)["http2MaxRequests"] = val
    
    # maxRequestsPerConnection: int32
    def set_http_max_requests_per_connection(self, i, val):
        self._get_http(i)["maxRequestsPerConnection"] = val
        
    # maxRetries: int32
    def set_http_max_retries(self, i, val):
        self._get_http(i)["maxRetries"] = val
    
    # idleTimeout : int (unit ms)
    def set_http_idle_timeout(self, i, val):
        self._get_http(i)["idleTimeout"] = str(val) + "ms"
        
    # useClientProtocol: bool
    def set_http_use_client_protocol(self, i, val):
        self._get_http(i)["useClientProtocol"] = val

In [37]:
ic = IstioConfig(config_file)

ic.set_tcp_keeplive_time(0, 150000)

ic.apply()

destinationrule.networking.istio.io/carts configured
destinationrule.networking.istio.io/carts-db unchanged
destinationrule.networking.istio.io/catalogue unchanged
destinationrule.networking.istio.io/catalogue-db unchanged
destinationrule.networking.istio.io/front-end unchanged
destinationrule.networking.istio.io/orders unchanged
destinationrule.networking.istio.io/orders-db unchanged
destinationrule.networking.istio.io/payment unchanged
destinationrule.networking.istio.io/queue-master unchanged
destinationrule.networking.istio.io/rabbitmq unchanged
destinationrule.networking.istio.io/session-db unchanged
destinationrule.networking.istio.io/shipping unchanged
destinationrule.networking.istio.io/user unchanged
destinationrule.networking.istio.io/user-db unchanged


In [42]:
ic = IstioConfig(config_file)
ic.size()

14

In [43]:
ic = IstioConfig(config_file)
ic.reset()
ic.save()